# PROJETO 1 - CLASSIFICADOR DE SENTIMENTO DO FILME JOKER
__________________________________________________________________________________________________________________

### Alunos:

Nome: Beatriz Cabral Fernandes 

Nome: Eduardo Ancona Mateus 


## 1. INTRODUÇÃO 

A proposta desse projeto é desenvolver um classificador que irá analisar como o público está reagindo ao premiado filme ***Joker***, do diretor *Todd Phillips*. Para isso, será utilizado como método o famoso algoritmo de Naive-Bayes.

## 2. DESENVOLVIMENTO E METODOLOGIA

A fim de simplificar a explicação do processo de desenvolvimento do projeto, ele será dividido em X etapas

### ETAPA 1 - Preparando o ambiente no Jupyter

Nessa etapa, serão baixadas e importadas todas as bibliotecas relevantes para o código, bem como implementadas todas as funções a serem utilizadas

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

#Importando as bibliotecas
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import Image
import re

#criando funcao de limpeza de caracteres
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

### ETAPA 2 - Autenticando no Twitter

Conta: `@datascience_dudle`


Aqui será feita a autenticação no twitter, a partir de um código obtido no próprio site. Apenas o detentor das chaves de acesso da conta no Twitter consegue rodar o cógigo, e por isso ele encontra-se comentado.

In [2]:
#leitura do arquivo no formato JSON
#with open('auth.pass') as fp:    
    
    #data = json.load(fp)

#Configurando a biblioteca. Não modificar
#auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
#auth.set_access_token(data['access_token'], data['access_token_secret'])

____________

### ETAPA 3 - Escolha de um produto e coleta das mensagens

No arquivo `Projeto1_Obtenção_dos_tweets.ipynb` foram coletados tweets relacionados ao filme ***Joker*** e salvos em na planilha do excel `Joker.xlsx`. Ao coletar os tweets com essa keyword, obtivemos 601 tweets para treinamento e 600 para teste. Em seguida foi feita a leitura dos arquivos.

In [3]:
# Fazendo a leitura da planilha de treinamento
excel = pd.read_excel('Joker.xlsx', sheet_name='Treinamento')
excel_teste = pd.read_excel('Joker.xlsx', sheet_name='Teste')

### ETAPA 4 - Classificação manual dos tweets

Vale ressaltar que o foco desse estudo é analisar o sentimento dos tweets relacionados ao premiado filme ***Joker***.
Desta forma, foram previamente estabelecidas 5 categorias para a classificação das mensagens:

* `P` - ***Crítica positiva*** – se a mensagem transmitida é uma crítica positiva;
* `N` - ***Crítica negativa*** – se a mensagem transmitida é uma crítica negativa;
* `I` - ***Irrelevante*** – se a mensagem transmitida estiver no contexto proposto, mas não for relevante para análise;
* `R` - ***Reação*** - se a mensagem transmitida trata-se de uma reação ou emoção em relação ao filme ou a uma cena;
* `F` - ***Fora do contexto*** - se a mensagem transmitida esta fora do contexto proposto.


Estabelecidas as categorias e selecionados os tweets, foi utilizada a base de treinamento, na qual as mensagens foram qualificadas manualmente no excel de acordo com a categoria mais adequada. Conforme mostra a tabela a seguir:

In [4]:
#Definindo 'Etiquetas' como categoria e renomeando as categorias (1=P, 2=N, 3=I, 4=R, 5=F)
excel.loc[:,'Etiquetas'] = excel['Etiquetas'].astype('category')
excel.Etiquetas.cat.categories = ['P','N','I','R','F']

excel.head()

,Treinamento,Etiquetas
0,mortos pelo regime genocida da china 100.000.0...,F
1,"@thiago_joker tempos de quarentena, sei lá né ...",F
2,se o governador do rio de janeiro candidatar p...,F
3,@jillajeeva332 @yutheesh0011 @joker_rowdy @pok...,F
4,galera só pra relembrar os filmes da dc de mai...,I


### ETAPA 5 - Montando o Classificador Naive-Bayes

### Breve explicação de *Naive Bayes*
O algoritmo de *Naive Bayes* é um classificador probabilístico baseado no teorema de Bayes, utilizado no processo de machine learning. O algoritmo supõe que uma característica independe da outra para acontecer, ou seja, mesmo na presença de uma característica particular em uma classe, isso não afeta na probabilidade de qualquer característica ocorrer. O teorema de bayes é escrito da seguinte forma:

![title](img/bayes.png)

* $P(A)$ - Probabilidade a priori
* $P(B)$ - Probilidade Total
* $P(A|B)$ - posteriori
* $P(B|A)$ - verossimilhanca


Esse método será utilizado no projeto, uma vez que permite calcular a probabilidade de uma mensagem receber diferentes classificações, por exemplo, dado um tweet, assumindo que as palavras nele não tem nenhuma relação entre si.

A partir do nosso modelo, poderíamos reescrever o teorema de bayes da seguinte forma:

# $P(C|P) = \frac{P(P|C).P(C)}{P(P)}$

A variável C é a classe variável que representa a categoria de um tweet, a partir das condições estabelecidas (probabilidade de ocorrência de uma palavra dada as condições). A variável P representa as palavras ocorridas nos tweets.


P pode ser:

![title](img/bayes2.png)

Substituindo P por cada uma das possíveis palavras, temos:

![title](img/bayes3.png)

Nesse momento queremos classificar se um tweet está dentro ou fora do contexto do filme Joker. Para tanto é necessário analisar analisar as probabilidades.


Ou seja, precisa decidir se:

## $P(Dentro do contexto|frase) > P(Fora do contexto|frase)$



Utilizando o teorema de Bayes, temos:

## $P(D|frase) = \frac{P(frase|D)P(D)}{P(frase)}$

e que:

## $P(F|frase) = \frac{P(frase|F)P(F)}{P(frase)}$



Para implementar esse algoritmo, uma nova tabela deve ser criada com as palavras e suas respectivas frequências relativas em cada uma das categorias. Porém antes disso, deverá ser feita uma limpeza das mensagens, removendo pontuações e caracteres que não convém a análise.

In [5]:
# Verificando categorias do meu dataset
list(excel)

['Treinamento', 'Etiquetas']

In [18]:
# Juntando todos os tweets da base de treinamento
texto_completo = cleanup(' '.join(excel.Treinamento)).split()
pd.Series(texto_completo).value_counts().head(15)


o        329
joker    324
de       262
que      227
a        226
e        219
é        175
do       162
https    147
//t      146
não      133
eu       105
rt        98
pra       95
com       77
dtype: int64

Fazendo uma breve análise dos 15 termos mais frequentes, observa-se que alguns desses termos não são significantes para a nossa análise, e portanto deve-se ser feita a limpeza para nao prejudicar o resultado/performance. Exemplo:
- https
- -
- rt
- /

In [54]:
def Frequencia(n):
    y = cleanup(' '.join(excel[excel.Etiquetas==n].Treinamento)).split()
    f_rel = pd.Series(y).value_counts(True)
    f_abs = pd.Series(y).value_counts()
    return y,f_rel,f_abs

### Definindo a probabilidade de cada uma das categorias

Observando a tabela a seguir, depreende-se que uma grande parte dos tweets fora do contexto do projeto. 

In [55]:
pd.Series(excel.Etiquetas).value_counts(True).to_frame()

,Etiquetas
F,0.464107
I,0.295492
R,0.110184
P,0.098497
N,0.031720


Dessa forma, como tambem para nao criar um classificador enviesado, vamos primeiro classificar os tweets em 2 principais categorias: como Dentro do contexto(soma das categorias P,N,R e I) ou Fora do contexto(categoria F).


Aqui, ainda utilizando a funcao anterior que além de limpar a base, retorna as frequencias de cada uma das categorias, podemos definir as seguintes probabilidades:

In [62]:
p, rel_p, abs_p = Frequencia('P')
n, rel_n, abs_n = Frequencia('N')
i, rel_i, abs_i = Frequencia('I')
r, rel_r, abs_r = Frequencia('R')
f, rel_f, abs_f = Frequencia('F')

#Definido a frequencias relativa e absoluta da nova categoria 'Dentro do contexto'
rel_d = rel_p + rel_n + rel_i + rel_r
abs_d = abs_p + abs_n + abs_i + abs_r
d = p + n + i + r 

#Definindo as probabilidades: soma = 1
prob_d = len(d)/len(p + n + i + r + f)
prob_f = len(f)/len(p + n + i + r + f)


In [63]:
tweet_teste = "joker"

probTweetDadoD = rel_d[tweet_teste.split()].prod()
probTweetDadoF = rel_f[tweet_teste.split()].prod()

print("Probabilidade de Tweet dado que é dentro do contexto: ", probTweetDadoD)
print("Probabilidade de Tweet dado que é fora do contexto: ", probTweetDadoF)

Probabilidade de Tweet dado que é dentro do contexto:  0.18705281607320318
Probabilidade de Tweet dado que é fora do contexto:  0.008324285197249367


In [64]:
probDDadoTweet = prob_d*probTweetDadoD
probFDadoTweet = prob_f*probTweetDadoF

print("Probabilidade de estar dentro do contexto dado que Tweet: ", probDDadoTweet)
print("Probabilidade de estar fora do contexto dado que Tweet: ", probFDadoTweet)

Probabilidade de estar dentro do contexto dado que Tweet:  0.0972323553832031
Probabilidade de estar fora do contexto dado que Tweet:  0.003997219325686479


In [66]:
class_tweet = 0
if probDDadoTweet>probFDadoTweet:
    class_tweet = 'D'
else:
    class_tweet = 'F'
class_tweet

'D'

### ETAPA 6 - Verificando a performance do classificador

## CONCLUSÃO

- Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a performance do seu classificador.
- Explique como são tratadas as mensagens com dupla negação e sarcasmo.
- Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?


Opcionalmente:

- Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.

*Ao se utilizar o próprio classificador para gerar mais amostras de treinamento, o classificador acaba ficando 
'viciado', ou seja, acaba-se por prejudicar os resultados obtidos, e eles acabam não tendo uma qualidade boa.
Seria o contrário de puxar mais tweets, que melhora a qualidade dos dados e dos resultados, por ter um espaço 
amostral muito maior.*



- Propor diferentes cenários de uso para o classificador Naive-Bayes. Pense em outros cenários sem intersecção com este projeto.
- Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer. Indique material de pesquisa sobre o assunto).